# PS6 - 26 Sep
## Updating your Julia - IJulia configuration
From time to time it is suggested to update your system. The code below should be run in a Julia terminal. This might take some time (depending on the amount of packages to be updated and your internet connection).

```Julia
using Pkg
Pkg.update()
```

## Disk indexing
* make a function that scans a folders' (and its subfolders) 
* alle folders en files inside the roots folder should be indexed
* write the structure to a text file, you should destinguish between files and folder (for readability)
* Use indentation in your text file to destinguish different levels
* test your application

An example output is given below:
```Julia
loc = "/Users/bart/Documents/KMS/ES313/ES313.jl/Exercises/spygames/"
listfolder(loc)
```

```
Analysis of /Users/bart/Documents/KMS/ES313/ES313.jl/Exercises/spygames/:

Folders in /Users/bart/Documents/KMS/ES313/ES313.jl/Exercises/spygames/:
	[HIDDEN].ipynb_checkpoints
	Files in /Users/bart/Documents/KMS/ES313/ES313.jl/Exercises/spygames//.ipynb_checkpoints:
		Warning-checkpoint.txt
	type 2
	type 3
Files in /Users/bart/Documents/KMS/ES313/ES313.jl/Exercises/spygames/:
	[HIDDEN].DS_Store
	Warning.txt


```

In [121]:
function listfolder(loc::String,level=0::Int)
    if isdir(loc)≠true
        error("Please enter a valid dir")        
    end
    fname = split(loc,"/")[end-1]
    # try to open the file, else append
    fstream = open(fname*".txt","w+")
    write(fstream,"Analysis of $loc:\n\n")
    crawfolder(loc,fstream)
    close(fstream)
end

function crawfolder(loc,stream::IO,level=0)
    sep = "\t"
    cont = readdir(loc)
    folders = [f for f in cont if isdir(loc*"/"*f)]
    files =  [f for f in cont if isfile(loc*"/"*f)]
    if length(folders) ≠ 0
        println(stream,"$(sep^level)Folders in $loc:")
        for folder in folders
            if folder[1]=='.'
                println(stream,"$(sep^(level+1))[HIDDEN]$folder")
            else
                println(stream,"$(sep^(level+1))$folder")
            end
            crawfolder(loc*"/$folder",stream,level+1)
        end
    end
    if length(files) ≠ 0
        println(stream,"$(sep^level)Files in $loc:")
        for file in files
            if file[1] == '.'
                println(stream,"$(sep^(level+1))[HIDDEN]$file")
            else
                println(stream,"$(sep^(level+1))$file")
            end
        end
    end
end

crawfolder (generic function with 3 methods)

In [124]:
loc = "/Users/bart/Documents/KMS/ES313/ES313.jl/Exercises/spygames/"
listfolder(loc)

### Working with types
* Write your own types holding (Lat,Long) coordinates of a point on earth. For ease of use, we will respect the following convention: $Lat\in\left[-90°,90°\right]$ and $Long\in\left[0°,360°\right[$
* The shortest path $P$ between two locations on a spere is calculated as follows ($R_{earth} \approx 6400$km):
\begin{equation}
\cos\left(p\right)=\sin\left(Lat_{A}\right)\sin\left(Lat_{B}\right)+\cos\left(Long_{A}\right)\cos\left(Long_{B}\right)\cos\left(\Delta Long\right)
\end{equation}
\begin{equation}
P = \cos^{-1}\left( p\right)*R_{earth}
\end{equation}

* write a function to determine the shortest path between two points
* write a function to determine the two closest or two furthest points from an array of points
* try your functions for some locations on the globe (generate random points)
* think about how you would like your points to look when printed

In [54]:
struct Point
    loc::String
    lat::Float64
    long::Float64
end

import Base.show
function show(io::IO,p::Point)
    write(io,"""$(p.loc) ($(lpad(round(p.lat,digits=8),5," "))° Lat, $(lpad(round(p.long,digits=8),5," "))° Long)""")
end

function degrad(angle::Real)::Real
    angle*pi/180
end

function sp(p1::Point,p2::Point) ::Float64
    R = 6400
    La = degrad(p1.lat)
    Lb = degrad(p2.lat)
    Δlong = degrad(p2.long - p1.long)
    acos(sin(La)*sin(Lb) + cos(La)*cos(Lb)*cos(Δlong)) * R
end

function sp(t::Tuple{Point,Point})
    sp(t[1],t[2])
end

function mindist(p::Array{Point,1})
    a = [(a,b) for a in p for b in p if a≠b]
    dist, ind = findmin(sp.(a))
    return a[ind],dist
end
                
function maxdist(p::Array{Point,1})
    a = [(a,b) for a in p for b in p if a≠b]
    dist, ind = findmax(sp.(a))
    return a[ind],dist
end

maxdist (generic function with 1 method)

In [57]:
Point("A",4.0,3.0)

A (  4.0° Lat,   3.0° Long)

In [58]:
lst = Array{Point,1}()
for i in 65:65+15
    push!(lst,Point("$(Char(i))",rand(-90:90),rand(0:360)))
end
lst

16-element Array{Point,1}:
 A ( 31.0° Lat, 139.0° Long)
 B ( 69.0° Lat, 293.0° Long)
 C (-44.0° Lat, 144.0° Long)
 D (-81.0° Lat,  59.0° Long)
 E (-34.0° Lat, 204.0° Long)
 F ( 68.0° Lat, 115.0° Long)
 G (-69.0° Lat, 341.0° Long)
 H (-56.0° Lat, 336.0° Long)
 I (-60.0° Lat,  40.0° Long)
 J ( 55.0° Lat, 168.0° Long)
 K ( 74.0° Lat, 332.0° Long)
 L ( -8.0° Lat, 130.0° Long)
 M (-74.0° Lat,  15.0° Long)
 N ( 20.0° Lat, 222.0° Long)
 O ( 34.0° Lat, 146.0° Long)
 P ( 24.0° Lat, 221.0° Long)

In [60]:
mindist(lst)

((N ( 20.0° Lat, 222.0° Long), P ( 24.0° Lat, 221.0° Long)), 458.6433859559117)

In [61]:
maxdist(lst)

((H (-56.0° Lat, 336.0° Long), J ( 55.0° Lat, 168.0° Long)), 19339.80511808115)